In [44]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
import splitfolders # Библиотека для разделения файлов картинок на train test
import matplotlib.gridspec as gridspec
import torch.nn.functional as F
cudnn.benchmark = True
torch.cuda.get_device_name(0)


'NVIDIA GeForce GTX 1650'

In [45]:
run = neptune.init_run(
    project="neas1231/Neas1231",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmY2VjMGIzOS01ZjI1LTQ1MTItODQxYi0zMjIyOWIwYWI0MzIifQ==",
)

# splitfolders.ratio(
#      "E:\datasets\dataset_noweather",
#      output='E:\datasets\split',
#      seed=42,
#      ratio=(0.7, 0.2, 0.1),
#      group_prefix=None,
#      move=False)


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((128 , 128)),   
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((128 , 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\split'
run["config/data_dir"] = data_dir

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
              for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) 
              for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

https://app.neptune.ai/neas1231/Neas1231/e/NEAS-106


In [46]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=14):
    since = time.time()
    #Сохраняем  лучшие веса 
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 99
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)


        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            # перенос  вычислений на куду 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                
                optimizer.zero_grad()

                #
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    acc = (torch.sum(preds == labels.data)) / len(inputs)
                    run["logs/training/batch/loss"].append(loss)
                    run["logs/training/batch/acc"].append(acc)

                    # считаем градиенты тольок если трейн данные 
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # стасистика  
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # сохраняем не только лучшие веса , но и акураси 
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Loss: {best_loss:4f}')
    

    model.load_state_dict(best_model_wts)
    return model

In [47]:
model_ft = models.efficientnet_b4(pretrained=True) 
num_ftrs = model_ft.classifier[1].in_features
# num_ftrs = model_ft.fc.in_features Для рес нета , но у эфинета другая архитектура 

model_ft.fc = nn.Linear(num_ftrs, len(class_names))
# cuda
model_ft = model_ft.to(device)
run["config/model"] = model_ft

criterion = nn.CrossEntropyLoss()
run["config/criterion"] = criterion

# оптимайзер  с лернинг рейт 
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
run["config/optimizer"] = optimizer_ft
run["parameters"] = model_ft.parameters()

# редактируем лернинг рейт каждые 7 шагов 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [48]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)
torch.save(model_ft,'efficientnet_b4-bestloss-no_weather_new.pth')
torch.save(model_ft.state_dict(), "weights-efficientnet_b4-bestloss-no_weather_new.pth")
run.stop()

Epoch 0/24
----------
train Loss: 1.4072 Acc: 0.6272
val Loss: 1.4350 Acc: 0.8178

Epoch 1/24
----------
train Loss: 0.6200 Acc: 0.8075
val Loss: 0.4950 Acc: 0.8551

Epoch 2/24
----------
train Loss: 0.4522 Acc: 0.8521
val Loss: 0.3852 Acc: 0.8895

Epoch 3/24
----------
train Loss: 0.3643 Acc: 0.8843
val Loss: 0.4239 Acc: 0.8820

Epoch 4/24
----------
train Loss: 0.3138 Acc: 0.8970
val Loss: 0.4461 Acc: 0.8839

Epoch 5/24
----------
train Loss: 0.2627 Acc: 0.9140
val Loss: 0.3919 Acc: 0.8990

Epoch 6/24
----------
train Loss: 0.2280 Acc: 0.9222
val Loss: 0.5135 Acc: 0.8777

Epoch 7/24
----------
train Loss: 0.2094 Acc: 0.9280
val Loss: 0.3795 Acc: 0.9037

Epoch 8/24
----------
train Loss: 0.1803 Acc: 0.9383
val Loss: 0.3762 Acc: 0.8994

Epoch 9/24
----------
train Loss: 0.1444 Acc: 0.9494
val Loss: 0.3729 Acc: 0.9112

Epoch 10/24
----------
train Loss: 0.1322 Acc: 0.9545
val Loss: 0.3559 Acc: 0.9112

Epoch 11/24
----------
train Loss: 0.1165 Acc: 0.9608
val Loss: 0.3564 Acc: 0.9136

Ep

In [43]:
import gc
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
gc.collect()

124

### Добавление слоёв

In [50]:
resnet = models.resnet18(pretrained = True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Identity()

class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.avgpool = nn.AvgPool2d(5,3)
        self.layer = nn.Linear(512, 512)
        self.dropout = nn.Dropout(0.5)
        self.sigm = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.exit_layer = nn.Linear(512, 11)

    def forward(self, x):
        x = self.dropout(x)
        x = self.layer(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.exit_layer(x)
        x = self.sigm(x)
        
        return x

In [51]:
net_add = net()
model = nn.Sequential(resnet, net_add)
model = model.to(device)

In [52]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)

Epoch 0/19
----------
train Loss: 2.1969 Acc: 0.1803
val Loss: 2.1272 Acc: 0.1539

Epoch 1/19
----------
train Loss: 2.2337 Acc: 0.1472
val Loss: 2.2149 Acc: 0.2227

Epoch 2/19
----------
train Loss: 2.2634 Acc: 0.1530
val Loss: 2.2959 Acc: 0.1174

Epoch 3/19
----------
train Loss: 2.3141 Acc: 0.1458
val Loss: 2.3124 Acc: 0.1376

Epoch 4/19
----------
train Loss: 2.3748 Acc: 0.1060
val Loss: 2.3979 Acc: 0.0709

Epoch 5/19
----------
train Loss: 2.3980 Acc: 0.0695
val Loss: 2.3979 Acc: 0.0735

Epoch 6/19
----------


KeyboardInterrupt: 

In [ ]:
model